In [ ]:
import numpy as np
import tensorflow as tf
import time
import json

a = tf.constant([1,3,4], tf.float32)
b = tf.constant([1.1,1.2,1.3], tf.float32)


b = tf.reduce_sum(tf.where(tf.abs(a) > 1, np.ones(a.shape), np.zeros(a.shape)))
with tf.Session() as sess:
    print(sess.run(b))



In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import fix_yahoo_finance as yf
import pandas 
import numpy as np
from datetime import datetime
from datetime import timedelta
import os.path

def downloadAndMerge(names):
    downloaded = []
    for n in names:
        csv = 'csv/' + n + '.csv'
        print(n)
        temp = None
        if os.path.isfile(csv):
            temp = pandas.read_csv(csv, index_col=0)
        else:
            temp = yf.download(n)
            temp.to_csv(csv)
        nv = temp[['Close']]
        nv.columns = [n]
        downloaded.append(nv)
    
    for i in downloaded:
        if i.index.dtype != 'datetime64[ns]':
            i.index = i.index.astype('datetime64')
    
    merged = pandas.concat(downloaded, axis=1, join='outer')
    x = merged.values 
    i = len(x) - 1
    while i > 0:
        if np.isnan(x[i]).any():
            break
        i-=1
    print(merged.columns[merged.iloc[i].isna()][0])
    print(merged.iloc[i].name)
    merged = merged.iloc[i+1:].T
    return downloaded, merged


def getD(df):
    base = 10
    a = df.iloc[:, :-1]
    b = df.iloc[:, 1:]
    b.columns = a.columns = range(df.shape[1] - 1)
    r = (b - a) / a
    d = np.concatenate((np.ones([r.shape[0], 1]) * base,
                        np.add.accumulate(r * base, axis=1) + np.ones(r.shape) * base), axis=1)
    return d 


def toWeek(df, start):
    idx = datetime.strptime(start, '%b %d %Y')
    week = []
    names = []
    while idx <= df.columns[-1]:
        names.append(idx)
        if idx in df.columns:
            week.append(df[idx].values)
        else:
            found = False
            for i in range(1, 7):
                nidx = idx + timedelta(days=(i*-1))
                if nidx in df.columns:
                    week.append(df[nidx].values)
                    found = True
                    break
            if not found:
                week.append(np.empty(df[idx].shape))
        idx = idx + timedelta(days=7)
    
    ndf = pandas.DataFrame(week).T
    ndf.index = df.index
    ndf.columns = names
    return ndf


def findDrop(s, debug=True):
    drop = 1 - s / np.maximum.accumulate(s)
    i = np.argmax(drop) # end of the period
    j = np.argmax(s[:i]) # start of period
    jump = 1
    while i + jump < len(s):
        if s[i + jump] >= s[j]:
            break
        jump += 1
        
    if debug:
        print('max drawback {:0.2f}%, from {:0.2f} to {:0.2f}'.format(drop[i] * 100, s[j], s[i]))    
        print('wait {} after max drop'.format(jump))
        print('drop lasting {}'.format(i - j))
        print('average drop', drop.mean())
    #pandas.DataFrame(drop).hist(bins=20)
    return drop[i] * 100, drop.mean()

In [ ]:
import numpy as np
import tensorflow as tf
import time
import json


# regression loss
def reg_loss(s, v):
    a = tf.get_variable("a", [1], tf.float32, initializer=tf.constant_initializer(0))
    x = np.arange(int(v.shape[1]))
    y = a * x + 10
    return tf.reduce_sum(tf.pow(s - y, 2)) / int(y.shape[0]) / 10000, a

def drop_loss(s, v):
    length = v.shape[1]
    mask = np.ones([length,length], dtype=bool)
    zero = np.zeros([length,length])
    for i in range(length):
        for j in range(i):
            mask[j][i] = 0
    
    b = tf.constant(mask)
    y = tf.tile(s, [length])
    y = tf.reshape(y, [length,length])
    y = tf.where(b, y, np.full([length,length], np.NINF))
    y = tf.reduce_max(y, axis=1)
    return tf.reduce_max(1 - s / y)

def anaTop(top, d, w, debug):
    x = np.abs(w.T[0])
    x.sort()
    selected = np.where(np.abs(w) > x[-1 * top])[0]
    if len(selected) > 0:
        t = d[selected] * w[selected]
        s = t.sum(axis=0)
        s = s / s[0] * 10
        drop, meandrop = findDrop(s, False)

        x = np.arange(s.shape[0])
        a, b = np.polyfit(x, s, 1)
        y = b + a * x
        if debug:
            print('slope {}, fit loss {}'.format(a, (np.power(s - y, 2)).sum() / y.shape[0]))
            print(merged.index[selected].tolist())
            print(w[selected].T)
        return drop, meandrop, selected, a
    else:
        return 1, 1, 1, 0

def getW(V, debug):
    w = tf.get_variable("weight", [V.shape[0], 1], tf.float32, initializer=tf.ones_initializer())
    v = tf.constant(V, tf.float32)
    s = tf.reduce_sum(w * v, axis=0)

    rl, a = reg_loss(s, v)
    losses = [drop_loss(s, v), rl]
    key_loss = sum(losses)
    w_loss = tf.pow(tf.reduce_sum(tf.pow(w, 2)) - 1, 2)
    std_loss = 1 - 20 * tf.nn.moments(tf.abs(w), axes=[0])[1][0]
    #std_loss = 0.01 * tf.cast(tf.reduce_sum(tf.where(tf.abs(w) > 0.1, np.ones(w.shape), np.zeros(w.shape))), tf.float32)
    total_loss = key_loss + w_loss + std_loss

    optimizer = tf.train.AdamOptimizer(0.002)
    train_op = optimizer.minimize(total_loss)

    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        start_time = datetime.now()
        for step in range(10000000):
            if step % 1000 == 0:
                output_w = sess.run(w)
                drop, meandrop, selected, slope = anaTop(6, V, output_w, debug)
                if (datetime.now() - start_time).seconds > 180:
                    print('took too much time, abort, drop {}, meandrop {}'.format(drop, meandrop))
                    return [], []
                if debug:
                    print(step)
                    print('a ', sess.run(a))
                    for l in losses:
                        print('losses ', sess.run(l))
                    print('not too big ', sess.run(w_loss))
                    print('look the same ', sess.run(std_loss))
                    print('total', sess.run(total_loss))
                    print('drop {} meandrop {}'.format(drop, meandrop))

                if drop < 4 and meandrop < 0.008:
                    return selected, (output_w[selected] / np.absolute(output_w[selected]).sum()).T[0]

            sess.run(train_op)

In [ ]:
#search = ['SPY','EEM','QQQ','HYG','IWM','XLF','VXX','UVXY','EFA','FXI','EWZ','TLT','GLD','FEZ','SMH','USO','SLV','GDX','XLI','XLE','DIA','XRT','XOP','KRE','AAPL','BAC','BABA','TSLA','FB','AMZN','GE','AMD','MU','C','INTC','AABA','EA','JPM','F','NXPI','NFLX','TWTR','MSFT','FOXA','WYNN','GM','CAT','WFC','X','T','JD','PBR','QCOM','USB','AA','WMT','AAL','BA','CMCSA','BIDU','KMI','MS','MET','FCX','NVDA','M','SQ','GS','V','BK','DB']
search = ['SPY','AAPL','AMZN','QQQ','NFLX','MSFT','KEYS','TSLA','IWM','GOOG','EEM','GOOGL','FB','BABA','AMD','NVDA','BRK-B','XLF','IVV','SBUX','EFA','XOM','BKNG','XLI','SQ','GE','BAC','IYR','XLU','INTC','JPM','PG','XLP','IBM','EWZ','ATVI','CVX','HYG','WFC','C','VXX','XLK','FXI','DIS','XLE','MU','MRK','PFE','V','UNH','EMB','MCD','ABBV','QCOM','T','LLY','BA','CSCO','KO','JNJ','LGND','IEMG','WMT','ADBE','UNP','GLD','VZ','DIA','CMCSA','LIN','HD','XLV','UTX','XOP','TQQQ','ORCL','CME','CRM','BIDU','HON','TWTR','MA','RHT','CAT','SMH','ABT','AVGO','PEP','SYY','TXN','COST','GS','MDY','GDX','LQD','MS','MMM','OXY','IEFA','NKE','VWO','KHC','TLT','VNQ','EOG','DWDP','AMGN','XLY','LMT','WBA','VOO','CVS','CL','MO','SLB','VTI','RTN','VEA','GD','BDX','XLB','GILD','SQQQ','TEVA','PM','GM','EL','EWJ','AAL','ISRG','MCHI','DGAZ','F','PYPL','EA','TVIX','IJH','CSX','IJR','COF','MDT','KRE','NOC','ECA','AMT','AGG','NEE','BMY','ACHC','NXPI','STX','MET','NWL','CELG','COP','PBR','MPC','CMG','SCHW','TSM','SPLK','JNK','TJX','LRCX','ULTA','BIIB','APC','LOW','STZ','HAL','LUV','EBAY','MTUM','EMR','WCG','AIG','BLK','CGC','TGT','PNC','MDLZ','ROKU','SHV','VLO','CCI','BAX','ALGN','XBI','BKLN','HRS','MAR','REGN','CI','DAL','INTU','NSC','ZTS','ANTM','WTW','USB','WDC','PLCE','BP','W','EW','AXP','SHW','COG','EXC','EQIX','EWT','WMB','FDX','NVS','AGN','HCA','ABMD','SPOT','CZR','PH','LLL','ACN','ETN','LVS','AMAT','SYMC','TDOC','IWD','USO','TMO','M','UPS','ILMN','GPC','VRSN','DRI','DUK','X','CHTR','USFD','UNG','JD','WDAY','FOXA','ADP','GIS','UGAZ','BBT','NOW','FDC','CAG','BIL','IEF','UVXY','TWLO','UAL','PSA','WYNN','SO','MYL','ANET','INDA','D','K','AKAM','APD','CIEN','ORLY','HES','GDXJ','AABA','KLAC','BK','ESRX','EWY','DE','IQV','PSX','EWW','KSS','NUE','FEZ','AET','WM','EXPE','XRT','FLOT','RSX','IWO','DXC','BBY','UAA','IP','TDG','ICE','CLX','YUM','FMC','CTSH','LULU','FTNT','PCG','DLR','KR','SAGE','MSI','IWB','RL','RACE','TNA','KEY','DHR','SWKS','PRU','VALE','SPGI','GRUB','CXO','CB','AAP','PXD','MCHP','SRE','DHI','CMI','NFX','IGV','BHC','DVN','TTWO','CHD','SDS','ROST','XLNX','IWF','SHY','MLM','AME','VTV','TYL','KMI','CNC','NTES','URI','STT','HLT','TMUS','VCSH','BX','MRO','DG','RDS-A','APTV','ADI','SPG','ABX','OMC','SWK','AMRN','VIG','WPC','VMC','BRK-A','APH','DXCM','NOV','ITUB','ADM','GOVT','QRVO','LYB','QID','WEC','TEL','PE','BSX','HPQ','SHOP','OLED','PBYI','VO','HUM','VIAB','ESS','UPRO','VST','FISV','KMB','CHK','AMP','XRAY','ET','EZU','HCP','MFC','LEN','CCL','PANW','FE','PAYX','XEL','PNW','JCI','SYK','MCK','FLR','NRZ','OHI','IWN','AZN','AEP','PEG','ED','EXAS','FSLR','ROP','SWN','TTD','CTL','ALL','GWW','COL','RRC','MCO','PPG','CMA','SYF','WP','OKE','HBAN','GDDY','STI','WRK','VFC','XPO','NBL','OIH','A','PGR','APA','VEEV','ENB','NRG','SPXU','ADSK','FANG','FAST','NCLH','CF','MHK','EQT','ALXN','CBS','MOH','IBB','SPLV','HRL','FLT','SBAC','BLL','MMI','SSO','AMLP','FOX','SCZ','MGM','CE','HOG','PLD','SRPT','SIRI','SJM','MAS','DLTR','LB','DPZ','ACWI','XEC','TPR','BND','RCL','WBT','IAC','DISCA','KORS','ITW','TLRY','MKC','EZA','EXPD','DTE','BBD','NTR','MTCH','IFF','CMS','RF','VRTX','GPN','AVB','TRV','NEM','AZO','MNST','DVA','MOS','ZBH','WLTW','MELI','HST','TRIP','ARNC','FITB','AES','SIVB','FL','DFS','MTB','SHPG','AFL','JBGS','TAP','TIP','SOXX','XLRE','LEA','BHGE','BURL','TRGP','BUD','NTAP','KDP','EIX','NUGT','LNC','HSIC','IR','REZI','WU','HSY','ECL','HYLB','MUB','LABU','CTXS','PBR-A','EPD','CRI','CLR','RPM','IDXX','IPG','SNA','SPTM','IAU','BG','ABEV','SPXL','JBHT','TAL','RH','ETR','LH','TIF','SH','HDV','MXIM','FCX','TSN','EXR','NVR','JWN','ON','NBIX','SPXS','VGK','BHP','TFI','GLW','CTAS','IVW','O','PACB','USMV','HBI','WELL','PHM','PPL','HAS','GG','CPT','GDI','QLD','GSK','EXEL','Z','VXF','FCE-A','MSG','WPX','HIG','WY','JNPR','ANSS','XLC','ZION','BHF','WLL','BF-B','WB','YUMC','VOD','VCIT','VTR','HIIQ','KWEB','BLUE','AR','CHRW','MTD','ROK','TOL','WEN','FTV','WAT','GGB','PTC','WEX','WSM','MRVL','DKS','SCHO','VUG','ULTI','IVZ','SEAS','MBB','EFX','EMN','ALK','CNP','BSV','ABC','CP','CFG','FIS','PAYC','MSCI','SCHF','IVE','GLNG','CDW','SEE','BR','RIG','CAH','VER','RJF','TZA','PII','CPB','TD','AOS','UA','SNAP','RIO','EWH','IQ','VRSK','HRC','HPE','LNG','OSK','CLF','EQR','DATA','VT','CTRP','MPW','RDS-B','DVMT','TRU','ETFC','ASML','KBE','FFIV','CWB','TSCO','PCAR','AMED','WHR','VAR','BXP','SU','OLLI','JAZZ','HFC','L','S']
downloaded, merged = downloadAndMerge(search)

In [ ]:
for i in range(len(downloaded)):
    if downloaded[i].columns[0] == 'SNAP':
        del downloaded[i]

In [ ]:
for i in downloaded:
    if i.index.dtype != 'datetime64[ns]':
        i.index = i.index.astype('datetime64')
        
merged = pandas.concat(downloaded, axis=1, join='outer')
merged.shape

In [ ]:
merged = toWeek(merged.T, 'Jan 01 1999').T

In [ ]:
step = 30 
pool = None
height = 1000000
def update(names, w, i):
    global pool
    global height 
    global step 
    entry = merged[names].iloc[i].values
    if pool is not None:
        now = merged[pool[0]].iloc[i].values
        old = pool[2]
        c = now / old - 1
        change = c * pool[1] * height
        height = change.sum() + height
        print('change {}, height {}'.format(c, height))

    pool = [names, w, merged[names].iloc[i].values]

def clear(i):
    global pool
    global height 
    global step 
    if pool is not None:
        now = merged[pool[0]].iloc[i].values
        old = pool[2]
        c = now / old - 1
        change = c * pool[1] * height
        height = change.sum() + height
        print('change {}, height {}'.format(c, height))
    pool = None

for i in range(30, merged.shape[0]):
    tf.reset_default_graph()
    temp = merged.iloc[i-step:i, :].dropna(axis=1).T
    print('i is {}, feed data from {} to {}'.format(i, temp.columns[0], temp.columns[-1]))
    d = getD(temp)
    selected, w = getW(d, False)
    if len(selected) > 0:
        names = temp.index[selected].tolist()
        print('decide to have {} {} at {}'.format(names, w, merged.index[i]))
        update(names, w, i)
    else:
        clear(i)

In [ ]:
height

In [ ]:
100000 * w / merged.iloc[:, 40].values[selected]

In [ ]:
d = getD(toWeek(merged, 'Nov 20 2015'))
print(d.shape)
V = d
V

In [ ]:
%matplotlib inline
pandas.DataFrame(d * np.array([[0.05311488], [0.81805724]])).sum(axis=0).plot()

In [ ]:
%matplotlib inline
s = df.values[0]
#s = 0.9 * d[0] + 0.1 * d[1]
findDrop(s)

# Load Data

In [ ]:
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt

base = 10
threshold = 0.6

df = pandas.read_csv('d.csv', index_col=0)
df = df.filter(regex='[\u4e00-\u9fa5]', axis=0)
a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns = range(df.shape[1] - 1)
r = (b - a) / a
p = 1 / np.linalg.norm(r, axis=1)
r = (r.T * p).T.values
l = r.shape[0]

correlation = np.ones([l, l])
for i in range(l):
    for j in range(i + 1, l):
        s = (r[i] * r[j]).sum()
        correlation[i][j] = s
        correlation[j][i] = s
correlation[np.abs(correlation) > threshold] = 0
correlation[np.logical_and(np.abs(correlation) <= threshold, correlation != 0)] = 1

d = np.concatenate((np.ones([r.shape[0], 1]) * base,
                        np.add.accumulate(r * base, axis=1) + np.ones(r.shape) * base), axis=1)

In [ ]:
selected, w = getW(V)

In [ ]:
s = (V[selected] * w).sum(axis=0)
findDrop(s)
x = np.arange(s.shape[0])
y = 0.1139553 * x + 10

s = (V[selected] * w).sum(axis=0)
x = np.arange(s.shape[0])
a, b = np.polyfit(x, s, 1)
y = b + a * x
print((np.power(s - y, 2)).sum() / y.shape[0])


plt.figure()
plt.plot(x, y)
plt.plot(x, s)
#plt.plot([i,j], [s[i], s[j]], 'o', color='Red', markersize=4)

In [ ]:
outer = pandas.concat(downloaded, axis=1, join='outer')
outer = outer[merged.index[selected]]
x = outer.values 
i = len(x) - 1
while i > 0:
    if np.isnan(x[i]).any():
        break
    i-=1
print(outer.columns[outer.iloc[i].isna()][0])
print(outer.iloc[i].name)
outer = outer.iloc[i+1:].T 
s = (getD(outer) * w).sum(axis=0)

In [ ]:
findDrop(s)

In [ ]:
with tf.Session() as sess:
    y = drop_loss(tf.constant([[0],[1]], tf.float32), tf.constant(V, tf.float32))
    print(sess.run(y))


In [ ]:
w = np.array([[0.54],[0.36]])
s = (w * V).sum(axis=0)
s = s / s[0] * 10
findDrop(s)
findDrop(d[469])
graph = np.concatenate([V, [s]], axis=0).T
pandas.DataFrame(graph).plot()

In [ ]:
latest = np.load('lasttime.npy')
print(latest[0:3])
print(latest[-3:])

for a in latest:
    if 469 in a[0]:
        print(a)
        break

dict = {}
for a in latest:
    for b in a[0]:
        if b in dict:
            dict[b] += 1
        else:
            dict[b] = 1
x = []
for key in dict:
    x.append([key, dict[key]])
x = sorted(x, key=lambda m: m[1], reverse=True)

for y in x:
    print(df.index[y[0]], y[0], y[1])

for a in latest[0:10]:
    print(list(df.index[a[0]]), a[1],a[2],a[3])


# Test By tiger data

In [ ]:
idx = [154, 184, 210, 428, 469, 538]
columns = list(np.array(df.index)[idx])
print(columns)

dates = df.columns.astype(np.float32)
startDate = dates[0]
dates = np.round((dates - startDate) / 3600 / 24 / 1000 / 7) + 1

coridx = np.zeros([len(idx), len(idx)])
for i in range(len(idx)):
    for j in range(i):
        coridx[i][j] = correlation[idx[i]][idx[j]]
        coridx[j][i] = correlation[idx[j]][idx[i]]
print(coridx)

# basic info
a = d[idx][:, 1:]
b = d[idx][:, :-1]
r = a - b
contri= np.abs(r).mean(axis=1)
contri = contri / contri.min()
print("contribution: {}".format(contri))

graph = pandas.DataFrame(d[idx]).T
graph.columns = columns 
graph.plot()
d[idx].sum(axis=0)[-1]

s = (d[idx]).sum(axis=0)
print("percentage of total value: {}".format(p[idx] / p[idx].sum() * 100))


# regression 
s = s / len(idx)
A, B = np.polyfit(dates, np.log(s), 1)
print('A,B', A, B)
y = np.exp(A * dates + B)
last = s[0] < y[0]
reg = 0
for i in range(1, len(s)):
    if (s[i] < y[i]) != last:
        reg += 1
        last = s[i] < y[i]
print('revisit times: ', reg)


# max drawback 
drop = 1 - s / np.maximum.accumulate(s)
i = np.argmax(drop) # end of the period
j = np.argmax(s[:i]) # start of period
print('max drawback {:0.2f}% {:0.2f} {:0.2f} {} {}'.format(drop[i] * 100, s[i], s[j], i, j))
print('average drop', drop.mean())
pandas.DataFrame(drop).hist(bins=20)


print(s[-1])
print(dates[-1])

plt.figure()
plt.plot(dates, y)
plt.plot(dates, s)
plt.plot([i,j], [s[i], s[j]], 'o', color='Red', markersize=4)

# Test by yahoo data

In [ ]:
from glob import glob
df = None
for f in glob('sss\\*.csv'):
    name = f.split('\\')[1].split('.')[0]
    if df is None:
        df = pandas.read_csv(f, index_col=0)[['Adj Close']]
        df.columns = [name]
    else:
        csv = pandas.read_csv(f, index_col=0)[['Adj Close']]
        csv.columns = [name]
        df = pandas.concat([df,csv], axis=1)
        
df = df.dropna().T
print(df.shape)
a = df.iloc[:, :-1]
b = df.iloc[:, 1:]
b.columns = a.columns
r = (((b - a) / a).T * p[idx] ).T
r = np.concatenate((np.ones([r.shape[0], 1]) * base,
                    np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1).T
r = pandas.DataFrame(r)
r.index = df.columns.astype(np.datetime64)
r.columns = df.index
r.plot(figsize=(16, 9))
print(r.shape)

y2 = r.sum(axis=1)
#plt.figure(figsize=(16, 9))
plt.plot(df.columns.astype(np.datetime64), y2 / len(idx))


# Test random output distribution

In [ ]:
import random

dist = []
n = 6
for i in range(100000):
    choice = random.sample(range(d.shape[0]), n)
    bad = True
    while bad:
        bad = False
        for k in range(len(choice)):
            for l in range(k):
                if np.abs(cor[choice[k]][choice[l]]) > 0.75:
                    bad = True
                    break
            if bad:
                break
        if bad:
            choice = random.sample(range(d.shape[0]), n)

    s = d[choice].sum(axis=0)
    s = s / n
    m = s.mean()
    s = s / m
    x = np.arange(len(s)) + 1
    a, b = np.polyfit(x, np.log(s), 1)
    y = np.exp(a * x + b)

    # reg times
    last = s[0] < y[0]
    reg = 0
    regDist = [0]
    for i in range(1, len(s)):
        if (s[i] < y[i]) != last:
            regDist.append(i)
            reg += 1
            last = s[i] < y[i]

    # reg distribution
    regDist = np.array(regDist)
    regStd = (regDist[1:] - regDist[:-1]).std()
    dist.append(regStd)
    
pandas.DataFrame(dist).hist(bins=1000)
pandas.qcut(dist,  [0, 0.05, 0.1, 0.25, 0.5, 0.75, 1])

# Load Weekly data and clean into d.csv

In [ ]:
%matplotlib inline
import pandas
from glob import glob
import numpy as np

df = None
for f in glob('D:\\work\\main\\IB\\*_week.csv'):
    if df is None:
        df = pandas.read_csv(f, index_col=0).T
    else:
        df = pandas.concat([df,pandas.read_csv(f, index_col=0).T], axis=1)

d = df.iloc[-112:-1,:]
d = d.dropna(axis=1,how='any').astype('float32').T
d.to_csv('d.csv')

# Load Daily data

In [ ]:
from glob import glob
%matplotlib inline
import pandas
import numpy as np
from matplotlib import pyplot as plt

daily = None
for f in glob('D:\\work\\main\\IB\\*_day.csv'):
    if daily is None:
        daily = pandas.read_csv(f, index_col=0).T
    else:
        daily = pandas.concat([daily,pandas.read_csv(f, index_col=0).T], axis=1)
daily.to_csv('day.csv')

# Test by day data

In [ ]:
from glob import glob
daily = pandas.read_csv('day.csv', index_col=0)[columns].T
print(columns)
moneyNeeded = 100 / (p[idx]/p[idx].sum()/daily.T.iloc[-1,:].values)[2]
tobuy = np.round(moneyNeeded * p[idx]/p[idx].sum() /  daily.T.iloc[-1,:].values)
moneyNeeded = tobuy * daily.T.iloc[-1,:].values
print('shares ', str(tobuy), tobuy.sum(), moneyNeeded)
moneyNeeded = moneyNeeded.sum()
print(moneyNeeded)

a = daily.iloc[:, :-1]
b = daily.iloc[:, 1:]
a.columns = b.columns
r = ((b / a - 1).T * p[idx] ).T
print(r.shape)

dates = daily.columns.astype(np.float32)
dates = np.round((dates - startDate) / 3600 / 24 / 1000) / 7 + 1
base = 10.4
r = np.concatenate((np.ones([r.shape[0], 1]) * base,
                    np.add.accumulate(r, axis=1) + np.ones(r.shape) * base), axis=1).T
r = pandas.DataFrame(r)
r.columns = columns
r.index = dates
r.plot(figsize=(16,9))

y = np.exp(A * dates + B) * moneyNeeded / 10
y2 = r.sum(axis=1) / len(columns) * moneyNeeded / 10
plt.figure(figsize=(16,9))
plt.plot(dates, y)
plt.plot(dates, y2)
#plt.plot(111, np.exp(A * 111 + B), marker='o', color='r', ls='')
#y2[-1] / y[-1] - 1